# yo-yo-maskr
## Part 1: Regex based anonymization

In [ ]:
import dill
import regex as re
from src.utils.regex import create_names_regex
from src.utils.trie import Trie

NAMES_FROM_PICKLE = False

if NAMES_FROM_PICKLE:
    with open('./data/first_names__regex.pkl', 'rb') as f:
        first_names = dill.load(f)
    with open('./data/last_names_trie_regex.pkl', 'rb') as f:
        last_names = dill.load(f)
    with open('./data/first_names_trie_regex.pkl', 'rb') as f:
        first_names_trie = dill.load(f)
    with open('./data/last_names_trie_regex.pkl', 'rb') as f:
        last_names_trie = dill.load(f)
else:
    with open('./data/first_names.txt') as f:
        first_names = [l.strip() for l in f.read().split('\n') if len(l.strip()) > 0]
    first_names_regex = create_names_regex(first_names)
    with open('./data/first_names_regex.pkl', 'wb') as f:
            dill.dump(first_names_regex, f)
    first_trie = Trie()
    for name in sorted([f for f in first_names if len(f) > 0], key=len, reverse=True):
        first_trie.add(name)
    first_trie_regex = re.compile(r'\b' + first_trie.pattern() + r'\b')
    with open('./data/first_names_regex.pkl', 'wb') as f:
        dill.dump(first_trie_regex, f)

    with open('./data/last_names.txt') as f:
        last_names = [l.strip() for l in f.read().split('\n') if len(l.strip()) > 0]
    last_names_regex = create_names_regex(last_names)
    with open('./data/last_names_regex.pkl', 'wb') as f:
        dill.dump(last_names_regex, f)
    last_trie = Trie()
    for name in sorted([l for l in last_names if len(l) > 0], key=len, reverse=True):
        last_trie.add(name)
    last_trie_regex = re.compile(r'\b' + last_trie.pattern() + r'\b')
    with open('./data/last_names_trie_regex.pkl', 'wb') as f:
        dill.dump(last_trie_regex, f)

In [ ]:
import regex as re
from tqdm.auto import tqdm
from src.utils.regex import anonymize_entities

with open('./sample_texts.txt') as f:
    sample_texts = f.read().split('\n')

last_trie_regex = re.compile(r'\b' + last_trie.pattern() + r'\b')
first_trie_regex = re.compile(r'\b' + first_trie.pattern() + r'\b')

result = [anonymize_entities(text, by_names='NAME', first_names=first_trie_regex, last_names=last_trie_regex) for text in tqdm(sample_texts[:])]
for i in range(len(result[:30])):
    print(result[i])